In [1]:
import os
import gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

/home/olli/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
env_name = 'CartPole-v0'
env = gym.make(env_name)

In [3]:
episodes = 5
for episode in range(1, episodes + 1):
    state = env.reset()
    done = False
    score = 0
    
    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score += reward
    print("episode and score:", episode, score)
env.close()

episode and score: 1 34.0
episode and score: 2 26.0
episode and score: 3 14.0
episode and score: 4 11.0
episode and score: 5 65.0


In [4]:
log_path = os.path.join('Training', 'Logs')

In [5]:
env = gym.make(env_name)
env = DummyVecEnv([lambda: env])
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

Using cpu device


In [6]:
model.learn(total_timesteps=20000)

2022-08-10 10:18:02.011405: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/olli/.local/lib/python3.8/site-packages/cv2/../../lib64:
2022-08-10 10:18:02.011450: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Logging to Training/Logs/PPO_1
-----------------------------
| time/              |      |
|    fps             | 263  |
|    iterations      | 1    |
|    time_elapsed    | 7    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 323         |
|    iterations           | 2           |
|    time_elapsed         | 12          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008338448 |
|    clip_fraction        | 0.0805      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.687      |
|    explained_variance   | 0.000812    |
|    learning_rate        | 0.0003      |
|    loss                 | 4.46        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0129     |
|    value_loss           | 51.3        |
-----------------------------------------
---

In [7]:
PPO_Path = os.path.join('Training', 'Savel Models', 'PPO_Model_Cartpole')
model.save(PPO_Path)

/home/olli/.local/lib/python3.8/site-packages/stable_baselines3/common/save_util.py:276: UserWarning: Path 'Training/Savel Models' does not exist. Will create it.
  warnings.warn(f"Path '{path.parent}' does not exist. Will create it.")


In [10]:
del model
model = PPO.load(PPO_Path, env=env)

In [11]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

/home/olli/.local/lib/python3.8/site-packages/stable_baselines3/common/evaluation.py:65: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


(200.0, 0.0)

In [12]:
env.close()

In [16]:
# testing
episodes = 5
for episode in range(1, episodes + 1):
    obs = env.reset()
    done = False
    score = 0
    
    while not done:
        env.render()
        action, _ = model.predict(obs)
        obs, reward, done, info = env.step(action)
        score += reward
    print("episode and score:", episode, score)
env.close()

episode and score: 1 [200.]
episode and score: 2 [200.]
episode and score: 3 [200.]
episode and score: 4 [200.]
episode and score: 5 [200.]


In [19]:
save_path = os.path.join('Training', 'Savel Models')

In [20]:
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold
stop_callback = StopTrainingOnRewardThreshold(reward_threshold=200, verbose=1)
eval_callback = EvalCallback(env, 
                                    callback_on_new_best=stop_callback, 
                                    eval_freq=10000, 
                                    best_model_save_path=save_path, 
                                    verbose=1)

In [21]:
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

Using cpu device


In [22]:
model.learn(total_timesteps=20000, callback=eval_callback)

Logging to Training/Logs/PPO_2
-----------------------------
| time/              |      |
|    fps             | 668  |
|    iterations      | 1    |
|    time_elapsed    | 3    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 497         |
|    iterations           | 2           |
|    time_elapsed         | 8           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.009851227 |
|    clip_fraction        | 0.11        |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.686      |
|    explained_variance   | -0.00755    |
|    learning_rate        | 0.0003      |
|    loss                 | 7.21        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0174     |
|    value_loss           | 51.8        |
-----------------------------------------
---

/home/olli/.local/lib/python3.8/site-packages/stable_baselines3/common/evaluation.py:65: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Eval num_timesteps=10000, episode_reward=181.60 +/- 36.80
Episode length: 181.60 +/- 36.80
-----------------------------------------
| eval/                   |             |
|    mean_ep_length       | 182         |
|    mean_reward          | 182         |
| time/                   |             |
|    total_timesteps      | 10000       |
| train/                  |             |
|    approx_kl            | 0.006715311 |
|    clip_fraction        | 0.057       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.609      |
|    explained_variance   | 0.277       |
|    learning_rate        | 0.0003      |
|    loss                 | 29          |
|    n_updates            | 40          |
|    policy_gradient_loss | -0.0148     |
|    value_loss           | 62.5        |
-----------------------------------------
New best mean reward!
------------------------------
| time/              |       |
|    fps             | 413   |
|    iterations      | 5     |
|    ti

In [27]:
net_arch = [dict(pi=[128,128,128,128], vf=[128,128,128,128])]

In [29]:
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path, policy_kwargs={'net_arch':net_arch})

Using cpu device


In [30]:
model.learn(total_timesteps=20000, callback=eval_callback)

Logging to Training/Logs/PPO_3
-----------------------------
| time/              |      |
|    fps             | 582  |
|    iterations      | 1    |
|    time_elapsed    | 3    |
|    total_timesteps | 2048 |
-----------------------------
----------------------------------------
| time/                   |            |
|    fps                  | 407        |
|    iterations           | 2          |
|    time_elapsed         | 10         |
|    total_timesteps      | 4096       |
| train/                  |            |
|    approx_kl            | 0.01459497 |
|    clip_fraction        | 0.213      |
|    clip_range           | 0.2        |
|    entropy_loss         | -0.681     |
|    explained_variance   | -0.00229   |
|    learning_rate        | 0.0003     |
|    loss                 | 4.04       |
|    n_updates            | 10         |
|    policy_gradient_loss | -0.024     |
|    value_loss           | 19.6       |
----------------------------------------
---------------------